# Synth Data Generating

In [ ]:
import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/GPLVM_project_code/')
from util_functions import *
import time

## Approach 1

In [ ]:
'''
w_n_C_total = 100 # totally 700 points for C
w_n_outputs = 50 # 1500
start_time = time.time()
w_X_true_1, w_C_total_1, w_sample_total_data_1, kernel_parameters_1 = tidily_sythetic_data_from_MOGP_smartly(n_C=w_n_C_total, n_X=w_n_outputs)
end_time = time.time()
print('Time for appraoch 1:', end_time - start_time)
shaped_data_1 = w_sample_total_data_1.reshape(w_n_outputs, w_n_C_total)
'''

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(shaped_data_1[17])

## Approach 2

In [ ]:
'''
start_time = time.time()
w_X_true_2, w_C_total_2, w_sample_total_data_2, kernel_parameters_2 = tidily_sythetic_data_from_MOGP(n_C=w_n_C_total, n_X=w_n_outputs)
end_time = time.time()
print('Time for appraoch 2:', end_time - start_time)
shaped_data_2 = w_sample_total_data_2.reshape(w_n_outputs, w_n_C_total)
'''

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(shaped_data_2[7])

In [ ]:
# torch.manual_seed(100)

In [ ]:
import pandas as pd
import torch
folder_path = '/Users/jiangxiaoyu/Desktop/All Projects/GPLVM_project_code/data/synth_regression/smartly_generated'
file_path = f'{folder_path}/ninputs_50_nlatents_100/target_data.csv'
data_raw = pd.read_csv(f'{file_path}')

In [ ]:
data_2d = torch.tensor(data_raw.to_numpy()).reshape(100, 50)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data_2d[65])

In [ ]:
import numpy as np

data = np.array([-0.028, -0.079, 0.0055])
print(data.mean())
print(data.std())

# Test something

In [ ]:
from gpytorch.kernels import ScaleKernel, RBFKernel, PeriodicKernel, MaternKernel
import torch

In [ ]:
kernel_1 = ScaleKernel(PeriodicKernel())
kernel_2 = ScaleKernel(RBFKernel())
kernel_3 = ScaleKernel(PeriodicKernel()) + ScaleKernel(RBFKernel())
kernel_4 = ScaleKernel(MaternKernel(nu=2.5))

In [ ]:
inputs_tensor = torch.tensor([0., 1., 2.]).reshape(-1,1)

In [ ]:
inputs_tensor

In [ ]:
kernel_1(inputs_tensor).to_dense().data

In [ ]:
kernel_2(inputs_tensor).to_dense().data

In [ ]:
kernel_3(inputs_tensor).to_dense().data

In [ ]:
kernel_4(inputs_tensor).to_dense().data